# Загрузка данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

data_dir = '/content/drive/My Drive/ProductBert'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
import string
import re
from tqdm import tqdm
import os
import numpy as np
from sklearn.metrics import accuracy_score

In [ ]:
data = pd.read_csv(os.path.join(data_dir, 'data.csv'))

In [ ]:
def preprocess_text(x):

    x = x.translate(str.maketrans('', '', string.punctuation))
    x = x.translate(str.maketrans('', '', string.digits))
    x = ''.join([w for w in x if not re.match(r'[A-Z]+', w, re.I)])

    return x.strip().lower()

In [ ]:
data['Наименование'] = data['Наименование'].apply(lambda x: preprocess_text(x))

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)

In [ ]:
import numpy as np

In [ ]:
data.groupby('Подкатегория_текст')['Наименование'].count().sort_values()

Подкатегория_текст
 Холодное оружие: луки спортивные массовые                                                                                                                                                                                                                                                                               1
Башмаки тормозных колодок подвижного состава магистральных железных дорог                                                                                                                                                                                                                                                1
Сыворотки прочие                                                                                                                                                                                                                                                                                                         1
Статические преобразователи для устр

In [ ]:
# data_sampled = pd.DataFrame()

# for code in data['Подкатегория_текст'].unique():
    
#     temp = data[data['Подкатегория_текст'] == code]
    
#     data_sampled = pd.concat([
#         data_sampled, 
#         temp.sample(min(temp.shape[0], 50))
#     ], ignore_index=True)

In [ ]:
# data_sampled.shape

In [ ]:
# data_sampled.groupby('Подкатегория_текст')['Наименование'].count().sort_values().describe()

In [ ]:
texts = list(data['Наименование'].values)

label_names = [str(x) for x in pd.get_dummies(data['Подкатегория_текст']).columns]
labels = pd.get_dummies(data['Подкатегория_текст']).values

labels = [np.where(x == 1)[0][0] for x in labels]

In [ ]:
# label_df = pd.DataFrame()
# label_df['name'] = label_names
# label_df = label_df.reset_index().rename(columns={'index': 'id'})
# label_df.to_csv(data_dir+'label_df.csv', index=False)

# Обучение модели

In [ ]:
# !pip install transformers

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizerFast, BertForSequenceClassification
  
model_version = 'sberbank-ai/ruBert-base'

tokenizer = BertTokenizerFast.from_pretrained(model_version)

Downloading:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/521 [00:00<?, ?B/s]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(texts, labels, test_size=0.1)

In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=100)
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=100)

In [ ]:
## PYTORCH CODE
import torch

class ProdDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ProdDataset(train_encodings, y_train)
val_dataset = ProdDataset(val_encodings, y_val)

In [ ]:
# model = BertForSequenceClassification.from_pretrained(model_version,
#                                                       num_labels=len(label_names))

model = BertForSequenceClassification.from_pretrained(data_dir+'/model5')

In [ ]:
## PYTORCH CODE
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=data_dir+'/logs',            # directory for storing logs
    # logging_steps=100,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    tokenizer=tokenizer
)

trainer.train()

***** Running training *****
  Num examples = 70620
  Num Epochs = 1
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1104


Epoch,Training Loss,Validation Loss
1,0.693900,0.775780


***** Running Evaluation *****
  Num examples = 7847
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-1104
Configuration saved in ./results/checkpoint-1104/config.json
Model weights saved in ./results/checkpoint-1104/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1104/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1104/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-1104 (score: 0.7757799625396729).


TrainOutput(global_step=1104, training_loss=0.6986201465993688, metrics={'train_runtime': 2268.9622, 'train_samples_per_second': 31.124, 'train_steps_per_second': 0.487, 'total_flos': 3641562262404000.0, 'train_loss': 0.6986201465993688, 'epoch': 1.0})

In [ ]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=100, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1).cpu().detach().numpy()[0]
    # executing argmax function to get the candidate label
    return np.where(probs == probs.max())[0][0]

In [ ]:
val_preds = []

for x_val in tqdm(X_val):
    val_preds.append(get_prediction(x_val))

100%|██████████| 7847/7847 [02:17<00:00, 57.19it/s]


In [ ]:
accuracy_score(y_val, val_preds)

0.8006881610806678

In [ ]:
trainer.save_model(data_dir+'/model6')

Saving model checkpoint to /content/drive/My Drive/ProductBert/model6
Configuration saved in /content/drive/My Drive/ProductBert/model6/config.json
Model weights saved in /content/drive/My Drive/ProductBert/model6/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/ProductBert/model6/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/ProductBert/model6/special_tokens_map.json


# Тестируем на сырой выборке

In [1]:
#!pip install transformers

  Using cached sacremoses-0.0.46-py3-none-any.whl (895 kB)
  Using cached huggingface_hub-0.1.2-py3-none-any.whl (59 kB)


In [ ]:
from transformers import BertTokenizerFast, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(data_dir+'/model6').to('cuda')

tokenizer = BertTokenizerFast.from_pretrained('sberbank-ai/ruBert-base')

In [ ]:
label_df = pd.read_csv(data_dir+'label_df.csv')

In [ ]:
label_df.to_csv('label_df.csv')

In [ ]:
test_data = pd.read_csv(os.path.join(data_dir, 'data.csv')).sample(50000)
test_data['Наименование'] = test_data['Наименование'].apply(lambda x: preprocess_text(x))
test_texts = list(test_data['Наименование'].values)

test_labels = pd.merge(pd.DataFrame(test_data['Подкатегория_текст']).rename(columns={'Подкатегория_текст': 'name'}), 
                       label_df, on=['name'], how='left').id.values

In [ ]:
test_preds = []
accuracys = []
i = 0

for x_test in tqdm(test_texts):

    if i % 1000 == 0:
        
        acc = round(accuracy_score(test_labels[:i], test_preds), 3)
        accuracys.append(acc)
        print('Accuracy on {} step – {}'.format(i, acc))

    i += 1
    test_preds.append(get_prediction(x_test))

  0%|          | 0/50000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  0%|          | 4/50000 [00:00<25:13, 33.04it/s]

Accuracy on 0 step – nan


  2%|▏         | 1005/50000 [00:20<17:02, 47.90it/s]

Accuracy on 1000 step – 0.918


  4%|▍         | 2004/50000 [00:40<15:50, 50.47it/s]

...

 96%|█████████▌| 48010/50000 [16:35<00:40, 49.34it/s]

Accuracy on 48000 step – 0.929


 98%|█████████▊| 49008/50000 [16:56<00:19, 50.06it/s]

Accuracy on 49000 step – 0.929


100%|██████████| 50000/50000 [17:16<00:00, 48.23it/s]


In [36]:
from sklearn.metrics import accuracy_score, f1_score, precision_score

print('accuracy_score on 50 000 samples:', accuracy_score(test_labels, test_preds), '\n')

print(f1_score(test_labels, test_preds, average='weighted'))

print(precision_score(test_labels, test_preds, average='weighted'))

accuracy_score on 50 000 samples: 0.92878 

0.9240579045440929
0.927017869699829


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Функция для инференса

In [ ]:
import string
import re
import pandas as pd


def preprocess_text(x):
    x = x.translate(str.maketrans('', '', string.punctuation))
    x = x.translate(str.maketrans('', '', string.digits))
    x = ''.join([w for w in x if not re.match(r'[A-Z]+', w, re.I)])
    return x.strip().lower()


def get_prediction(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=100, return_tensors='pt').to('cpu')
    outputs = model(**inputs)
    probs = outputs[0].softmax(1).cpu().detach().numpy()[0]
    return np.where(probs == probs.max())[0][0], probs.max()


def bert_inference(data):
    texts = list(data['Наименование'].apply(lambda x: preprocess_text(x)).values)
    test_preds = []
    for text in tqdm(texts):
        pred_label, pred_prob = get_prediction(text)
        pred_label = label_df[label_df.id == pred_label].name.iloc[0]
        pred_prob = round(pred_prob, 3)
        test_preds.append([pred_label, pred_prob])
    data[['предсказанная_Подкатегория_текст', 'Вероятность']] = test_preds
    return data


from transformers import BertTokenizerFast, BertForSequenceClassification

path_to_pretrained_model = data_dir+'/model6'

model = BertForSequenceClassification.from_pretrained(path_to_pretrained_model)
tokenizer = BertTokenizerFast.from_pretrained('sberbank-ai/ruBert-base')

label_df = pd.read_csv(data_dir+'label_df.csv')

In [ ]:
bert_inference(test_data.sample(1000))

100%|██████████| 1000/1000 [02:35<00:00,  6.43it/s]


,Наименование,Код_ЕП_РФ_подкатегория,Подкатегория_текст,Код_подкатегория,предсказанная_Подкатегория_текст,Вероятность
273214,посуда из пластмасс для взрослых в том числе с...,2293.0,Изделия культурно-бытового назначения и хозяйс...,71,"Посуда, в том числе одноразового применения (к...",0.588
24082,микардис таблетки мг шт упаковки ячейковые к...,9300.1,"Лекарственные средства, зарегистрированные в у...",156,"Лекарственные средства, зарегистрированные в у...",0.999
7918,цитрамонлект таблетки шт упаковки ячейковые к...,9300.1,"Лекарственные средства, зарегистрированные в у...",156,"Лекарственные средства, зарегистрированные в у...",0.999
44548,алфлутоп раствор для инъекций мл ампулы темно...,9300.1,"Лекарственные средства, зарегистрированные в у...",156,"Лекарственные средства, зарегистрированные в у...",0.998
241,карбамазепин таблетки мг шт упаковки ячейков...,9300.0,"Медикаменты, химико-фармацевтическая продукция...",189,"Лекарственные средства, зарегистрированные в у...",0.833
...,...,...,...,...,...,...
230988,сухие гранулированные корма для непродуктивных...,9219.1,Корма животного происхождения (включая корма д...,145,Корма животного происхождения (включая корма д...,0.558
209613,зажигалки кроме питаемых от сети марка «» « » ...,9692.2,Зажигалки (кроме питаемых от сети),62,Зажигалки (кроме питаемых от сети),0.990
101803,диклофенак гель для наружного применения г ...,9300.1,"Лекарственные средства, зарегистрированные в у...",156,"Лекарственные средства, зарегистрированные в у...",0.982
61317,сенаде таблетки мг шт упаковки ячейковые кон...,9300.1,"Лекарственные средства, зарегистрированные в у...",156,"Лекарственные средства, зарегистрированные в у...",0.999


# Интерпретация модели

In [ ]:
!pip install transformers-interpret

In [ ]:
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers_interpret import SequenceClassificationExplainer

model_ = BertForSequenceClassification.from_pretrained(data_dir+'/model2')

tokenizer = BertTokenizerFast.from_pretrained('sberbank-ai/ruBert-base')

In [ ]:
cls_explainer = SequenceClassificationExplainer(model_, tokenizer)
word_attributions = cls_explainer(X_val[0])

In [ ]:
word_attributions

[('[CLS]', 0.0),
 ('диф', -0.2746449512509223),
 ('##люка', -0.007256275827461156),
 ('##н', -0.01630014240946843),
 ('порошок', 0.058419181658606885),
 ('для', -0.08696114882494556),
 ('приготовления', -0.12324108376767691),
 ('су', -0.007056759491409794),
 ('##спен', -0.03700200667123896),
 ('##зии', 0.0277030457814121),
 ('для', 0.0071234709107578995),
 ('приема', -0.024686156975558548),
 ('внутрь', 0.033667412934518644),
 ('мг', 0.26660069249261786),
 ('мл', 0.07303240504601471),
 ('флакон', 0.22518619195498454),
 ('##ы', -0.012670780749270454),
 ('в', -0.07372753425691753),
 ('комплекте', -0.010989733368204942),
 ('с', -0.03141239640383053),
 ('лож', 0.09436570691010775),
 ('##ко', -0.0066625076987905365),
 ('##и', -0.004980702077321382),
 ('мерно', -0.049673472264284035),
 ('##и', 0.04792532995109837),
 ('пачки', 0.2715367688769194),
 ('картон', 0.2534528970195248),
 ('##ные', 0.07552983380568992),
 ('год', 0.5431859901912499),
 ('##ен', 0.09807008631661074),
 ('до', 0.3620264841

In [ ]:
cls_explainer.predicted_class_name

'LABEL_156'

In [ ]:
cls_explainer.visualize("ruBert.html")

In [ ]:
label_names[156]

'Лекарственные средства, зарегистрированные в установленном порядке и внесенные в государственный реестр, состоящие из смешанных и несмешанных продуктов для использования в терапевтических или профилактических целях, расфасованные в виде дозированных лекарственных форм или в упаковки для розничной продажи'

In [ ]:
y_val[0]

156